In [ ]:
# faq_chatbot.py

import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity # pyright: ignore[reportMissingModuleSource]

# ---------------------------
# Step 1: Collect FAQs
# ---------------------------
faq_data = [
    {"question": "How can I track my order?",
     "answer": "You can track your order using the tracking link sent to your email."},

    {"question": "What is your return policy?",
     "answer": "Items can be returned within 30 days of purchase with the original receipt."},

    {"question": "Do you offer international shipping?",
     "answer": "Yes, we ship to most countries worldwide. Shipping fees may vary."},

    {"question": "How do I cancel my order?",
     "answer": "Orders can be cancelled within 12 hours of placement via your account."},

    {"question": "What payment methods are accepted?",
     "answer": "We accept Visa, MasterCard, PayPal, and Apple Pay."},

    {"question": "Can I change my delivery address?",
     "answer": "Yes, before shipping. Go to your account and update the delivery details."},

    {"question": "Is there a warranty on products?",
     "answer": "Yes, all products come with a one-year manufacturer warranty."},

    {"question": "How long does delivery take?",
     "answer": "Delivery typically takes 3-7 business days depending on your location."},

    {"question": "Can I get a refund?",
     "answer": "Yes, refunds are processed once the returned item is received and inspected."}
]

# ---------------------------
# Step 2: Preprocess with SpaCy
# ---------------------------
print("🔄 Loading SpaCy model for preprocessing...")
nlp = spacy.load("en_core_web_sm")

def preprocess(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Preprocess questions
processed_questions = [preprocess(faq["question"]) for faq in faq_data]

# ---------------------------
# Step 3: Vectorize FAQ questions using TF-IDF
# ---------------------------
vectorizer = TfidfVectorizer()
faq_vectors = vectorizer.fit_transform(processed_questions)

# ---------------------------
# Step 4: Match question with cosine similarity
# ---------------------------
def get_best_answer(user_input):
    processed_input = preprocess(user_input)
    user_vector = vectorizer.transform([processed_input])
    similarities = cosine_similarity(user_vector, faq_vectors)
    best_match_idx = similarities.argmax()
    best_score = similarities[0][best_match_idx]

    if best_score < 0.3:
        return "Sorry, I couldn't find a relevant answer. Please try rephrasing your question."

    return faq_data[best_match_idx]["answer"]

# ---------------------------
# Step 5: Console-based Chatbot
# ---------------------------
def run_chatbot():
    print("🤖 Welcome to the FAQ Chatbot!")
    print("Ask your question about orders, shipping, payments, etc.")
    print("Type 'exit' to end the chat.\n")

    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit']:
            print("Bot: Thank you! Have a great day! 👋")
            break

        response = get_best_answer(user_input)
        print("Bot:", response)

# ---------------------------
# Run the chatbot
# ---------------------------
if __name__ == "__main__":
    run_chatbot()


🔄 Loading SpaCy model for preprocessing...
🤖 Welcome to the FAQ Chatbot!
Ask your question about orders, shipping, payments, etc.
Type 'exit' to end the chat.



In [26]:
%%html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Simple Translator</title>
  <style>
    body {
      font-family: Arial, sans-serif;
      max-width: 600px;
      margin: 50px auto;
      padding: 20px;
      border: 1px solid #ccc;
      border-radius: 8px;
    }
    textarea {
      width: 100%;
      height: 100px;
      margin-top: 10px;
    }
    select, button {
      padding: 10px;
      margin-top: 10px;
      margin-right: 10px;
    }
    #output {
      background: #f4f4f4;
      padding: 15px;
      margin-top: 20px;
      border-radius: 5px;
    }
    #controls {
      margin-top: 10px;
    }
  </style>
</head>
<body>

  <h2>🌍 Language Translator</h2>

  <label>Enter Text:</label>
  <textarea id="inputText" placeholder="Type something to translate..."></textarea>

  <div>
    <label>From:</label>
    <select id="fromLang">
      <option value="auto">Auto Detect</option>
      <option value="en">English</option>
      <option value="es">Spanish</option>
      <option value="fr">French</option>
      <option value="de">German</option>
      <option value="hi">Hindi</option>
    </select>

    <label>To:</label>
    <select id="toLang">
      <option value="en">English</option>
      <option value="es">Spanish</option>
      <option value="fr">French</option>
      <option value="de">German</option>
      <option value="hi">Hindi</option>
    </select>
  </div>

  <button onclick="translateText()">Translate</button>

  <div id="output">
    <strong>Translated Text:</strong>
    <p id="translatedText"></p>

    <div id="controls">
      <button onclick="copyText()">📋 Copy</button>
      <button onclick="speakText()">🔊 Speak</button>
    </div>
  </div>

  <script>
    async function translateText() {
      const text = document.getElementById("inputText").value;
      const from = document.getElementById("fromLang").value;
      const to = document.getElementById("toLang").value;

      if (!text.trim()) {
        alert("Please enter text to translate.");
        return;
      }

      const url = `https://translate.googleapis.com/translate_a/single?client=gtx&sl=${from}&tl=${to}&dt=t&q=${encodeURIComponent(text)}`;

      try {
        const res = await fetch(url);
        const data = await res.json();
        const translated = data[0].map(item => item[0]).join("");
        document.getElementById("translatedText").innerText = translated;
      } catch (err) {
        console.error(err);
        document.getElementById("translatedText").innerText = "Translation failed.";
      }
    }

    function copyText() {
      const text = document.getElementById("translatedText").innerText;
      navigator.clipboard.writeText(text).then(() => {
        alert("Text copied to clipboard!");
      });
    }

    function speakText() {
      const text = document.getElementById("translatedText").innerText;
      const toLang = document.getElementById("toLang").value;
      const utterance = new SpeechSynthesisUtterance(text);
      utterance.lang = toLang;
      speechSynthesis.speak(utterance);
    }
  </script>

</body>
</html>